In [1]:
import os
import pandas as pd
from params import *

In [2]:
data = pd.read_excel(
    f'{RAW_DATA_DIR}/dwelling-stock/subnationaldwellingstockbytenure2021.xlsx',
    sheet_name='1b',
    header=3,
)
data.columns = pd.Index(data.columns.to_series().str.strip())
data.set_index(data.columns.to_list()[2:4], drop=True, inplace=True)
data = data.rename_axis(index={
    'Local authority code': 'geography_code', 
    'Local authority name': 'geography_name'
    })

In [3]:
data = data.drop(columns=['Region code', 'Region name']).melt(ignore_index=False)

In [4]:
data.loc[:, ['date', 'type_of_tenure']] = data.variable.str.split('\s+', n=1).to_list()
data = data.drop(columns='variable')

In [5]:
data['variable_name'] = 'Number of dwellings'

In [6]:
data = data.set_index(['date', 'type_of_tenure', 'variable_name'], append=True)

In [7]:
PARQUET_FILE = f'../../../data-mart/dwelling-stock/dwelling-stock.parquet'
os.makedirs(os.path.dirname(PARQUET_FILE), exist_ok=True)
data.to_parquet(PARQUET_FILE)